# Notebook for loading data and beginning machine learning

Research: 
- journal: http://iraj.in/journal/journal_file/journal_pdf/1-10-139036756510-16.pdf
    - Overview of supervised learning algorithms

## Load data

In [69]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [149]:
# load from data file

path = "processed_images/"
name = "data.txt"
filename = path + name

data = pd.read_csv(filename, delimiter = ",", engine='python')

# don't need these right now...
#img_nums = data.loc[:,'img_num']
#X = data.loc[:,'x']
#Y = data.loc[:,'y']
#labels = data.loc[:,'label']

#coordinates = []
#for i in range(0, len(X)):
#    coordinates.append([X[0], Y[0]])

images = []

for i in range(0, len(data)):
    img_num = img_nums[i]
    img_file = "{}.png".format(img_num)
    img_name = path+img_file

    img = cv2.imread(img_name)
    images.append(img)
    
# add images to dataframe for ease later
data['image'] = pd.Series(images, index=data.index)

# CHECK:
#img = data['image'].iloc[0]
#plt.imshow(img)
#print(data)
#print(labels)
#print(len(images))

## Create training and testing sets

In [150]:
## split into training and testing sets
num_samples = data.shape[0]
training_size = .5
testing_size = 1 - training_size

# generate random sample from data for training set, and make testing set all else
msk = np.random.rand(num_samples) < training_size
training_data = data[msk]
testing_data = data[~msk]

#print(training_set)
#print(testing_set)

## Pre-process data with Principle Component Analysis
to extract meaningful features from data. Images are too big!

In [70]:
# imports
from sklearn.decomposition import PCA

In [190]:
# mess with images
training_images = np.array(training_data.loc[:,'image'])#.tolist()
print(training_images.shape)
img = training_images[0]
print(type(img[0]))
print(training_images[0:5][0])


#img = training_images.iloc[2]
#pca = PCA(n_components=1)
#pca.fit(training_images)

#print(training_images.shape)
#print(training_images[0])
#plt.imshow(img)
#print(training_images.iloc[2])
#print(img.shape)

type(training_images)

(285,)
<class 'numpy.ndarray'>
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[ 33  32  33]
  [ 16  20  16]
  [ 74  73  74]
  [ 74  77  74]
  [  0   0   0]
  [ 33  36  33]]

 [[  8  12   8]
  [ 33  36  33]
  [ 41  45  41]
  [ 41  40  41]
  [ 41  40  41]
  [ 33  40  41]]

 [[ 49  49  49]
  [ 33  36  33]
  [ 58  61  58]
  [ 66  69  66]
  [ 58  65  66]
  [ 49  53  49]]

 [[ 41  45  41]
  [ 33  32  33]
  [156 154 156]
  [255 255 255]
  [ 41  45  41]
  [ 58  61  58]]]


numpy.ndarray

In [184]:
test = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
test.shape
print(test)

[[-1 -1]
 [-2 -1]
 [-3 -2]
 [ 1  1]
 [ 2  1]
 [ 3  2]]
